# Trabajo colaborativo

El flujo de trabajo que hasta ahora hemos visto es:

```
    > git init                  # inicializa un repositorio
    > git add <archivo>         # da a conocer a git un archivo nuevo o un cambio en un archivo
    > git status                # resume los cambios actuales
    > git commit -m "mensaje"   # saca una foto instantánea del estado actual del proyecto 
    > git log                   # muestra la bitácora del proyecto

```
Otro comando que quizá usaron en la tarea fueron:
```
    > git push                  # sube los cambios a un repositorio central (local o remoto)
```

(Por ahora ">" indicará la línea de comandos.)

Ahora veremos un modelo posible de *colaboración*, que iremos complicando poco a poco. Además profundizaremos en el uso del push. 

La situación que consideraremos es la siguiente: Alicia (*Alice*) y Beto (*Bob*) colaboran en un proyecto (el que acabamos de clonar). Ambos tienen la misma versión del código. 

**Alicia:**
Alicia edita el archivo `archivo.txt`, y hace algún cambio que le parece conveniente. Siguiendo el esquema de trabajo que describimos arriba, Alicia sube los cambios a su repositorio local con `git add` y `git commit`, y finalmente los sube al repositorio central: `git push`.

**Beto:**
Beto, por su parte y de manera independiente, hace cambios *al mismo archivo* en que trabajó Alicia. De la misma manera que lo hizo Alicia, Beto actualiza su repositorio local (`git add` y `git commit`) y los sube al repositorio que comparten con `git push`.

*Sin embargo*, como él editó el mismo archivo en el que Alicia hizo cambios, pero usando una versión atrasada que *no* incluye los cambios de Alicia, entonces `git` detecta que hubo cambios divergentes entre la versión local de Beto, en la *rama* `master`, y la del repositorio remoto `origin/master`. Esto hace que `git` no permita subir los cambios que propone Beto, hasta que Beto resuelva los *conflictos* que hayan surgido.

## Enviando los commits a otro repositorio

Vamos a tratar de producir una situación controlada en la que tengamos dicha
situación. 

Para esto vamos a crear un repositorio nuevo. Este repositorio no será de 
trabajo. Solamente tendrá la información de los archivos, y los comits. 
Para esto ejecutamos, desde un directorio de prueba (por ejemplo `/tmp` o `~/Desktop`

In [1]:
;cd ~/Desktop

/mnt/homes/home/david/Desktop


In [2]:
;git init --bare original.git

Initialized empty Git repository in /mnt/homes/home/david/Desktop/original.git/


Los contenidos del directorio son los contenidos del directorio ´.git´ de otros
repositorios normales. Esto es por la opción ´--bare´.   
Después clonamos este repositorio "central" a otro directorio

In [3]:
;ls -la original.git

total 2156
drwxrwxr-x  6 david david    4096 feb 24 15:02 .
drwxrwxr-- 27 david david 2170880 feb 24 15:02 ..
-rw-rw-r--  1 david david      66 feb 24 15:02 config
-rw-rw-r--  1 david david      73 feb 24 15:02 description
-rw-rw-r--  1 david david      23 feb 24 15:02 HEAD
drwxrwxr-x  2 david david    4096 feb 24 15:02 hooks
drwxrwxr-x  2 david david    4096 feb 24 15:02 info
drwxrwxr-x  4 david david    4096 feb 24 15:02 objects
drwxrwxr-x  4 david david    4096 feb 24 15:02 refs


In [4]:
;git clone original.git alice

Cloning into 'alice'...
done.


Pero vemos que el repositorio nuevo es normal:

In [5]:
;cd alice

/mnt/homes/home/david/Desktop/alice


In [6]:
;ls -alp 

total 2132
drwxrwxr-x  3 david david    4096 feb 24 15:02 ./
drwxrwxr-- 28 david david 2170880 feb 24 15:02 ../
drwxrwxr-x  6 david david    4096 feb 24 15:02 .git/


Ahora creamos un archivo de prueba ahi mismo y hacemos el procedimiento usual para hacer un commit.

In [7]:
;echo "iniciamos este archivo con perros y gatos" >> inicio.txt

In [8]:
;git add . && git commit -m"Commit inicial"

[master (root-commit) 600fbcc] Commit inicial
 1 file changed, 1 insertion(+)
 create mode 100644 inicio.txt


Ahora, dado que clonamos el repositorio de otro, vamos a enviar los cambios (la creación del archivo `inicio.txt`) al respositio central:

In [9]:
;git push origin master

To /mnt/homes/home/david/Desktop/original.git
 * [new branch]      master -> master


La sintaxis de este comando es `git push repository branchname`. En este caso, `repository` es la dirección o referencia de algun repositorio. Esta puede ser, por `git@github.com:git/git.git`, pero por lo generalusamos alguna que esté determinada por defecto. Por ejemplo, vamos a usar  
`origin`  
que de acuerdo a 

In [10]:
;git remote -v

origin	/mnt/homes/home/david/Desktop/original.git (fetch)
origin	/mnt/homes/home/david/Desktop/original.git (push)


es `/home/carlosp/Desktop/tmp/original.git` (en mi caso). El `branchname` es la rama. Hasta ahora, no hemos creado ramas, por lo que nos referiremos a la rama principal, que se llama `master`. Después de el primer comando `git push origin master` podemos abreviarlo como `git push`.

Ahora creamos un archivo sin importancia, y modificamos nuestro archivo importante:


In [11]:
;echo "informacion trivial" >> algo.tmp && echo "mas informacion" >> inicio.txt

In [12]:
;git status

On branch master
Your branch is up-to-date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   inicio.txt

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	algo.tmp

no changes added to commit (use "git add" and/or "git commit -a")


Viendo el `status` notamos que `git` sabe que hemos cambiado el archivo `inicio.txt` y hemos creado un nuevo archivo `algo.tmp`. Supongamos que por error añadimos este último archivo sin importancia:

In [13]:
;git add . && git status

On branch master
Your branch is up-to-date with 'origin/master'.

Changes to be committed:
  (use "git reset HEAD <file>..." to unstage)

	new file:   algo.tmp
	modified:   inicio.txt



Entonces, el archivo `algo.tmp` se encuentra listo para que los sigamos. Sin embargo, no es lo que queremos. El comando 

In [14]:
;git rm --cached algo.tmp

rm 'algo.tmp'


hace que lo borremos de lo que queremos en el commit. Si lo incluimos en el `.gitignore`, entonces vemos que ya no aparece en el status:

In [15]:
;echo "algo.tmp" >> .gitignore

In [16]:
;git status && git add .  && git commit -m"Se mejora el gitignore y se anade mas informcion a inicio.txt" && git push

On branch master
Your branch is up-to-date with 'origin/master'.

Changes to be committed:
  (use "git reset HEAD <file>..." to unstage)

	modified:   inicio.txt

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	.gitignore

[master 66ef914] Se mejora el gitignore y se anade mas informcion a inicio.txt
 2 files changed, 2 insertions(+)
 create mode 100644 .gitignore


To /mnt/homes/home/david/Desktop/original.git
   600fbcc..66ef914  master -> master


## Interaccion y errores controlados

Comenzamos creando otro repositorio en el que otro personaje (Bob) va a trabajar.

In [17]:
;cd .. 

/mnt/homes/home/david/Desktop


In [18]:
;git clone original.git bob

Cloning into 'bob'...
done.


In [19]:
;cd bob

/mnt/homes/home/david/Desktop/bob


Para facilitar un poco la lectura de los logs, vamos a poner otro nombre para el usuario:

In [20]:
;git config user.name Bob

## Bob hace un cambio y Alice lo ve

Ahora vamos a crear un nuevo archivo, y a enviarlo.

In [21]:
;echo "esta es una idea de Bob" >> idea_bob.txt

In [22]:
;git add . && git commit -m"Nueva idea" && git push && git log

[master ce94b43] Nueva idea
 1 file changed, 1 insertion(+)
 create mode 100644 idea_bob.txt
commit ce94b43bab15d6408ec4d3e78233a65eec2a69e6
Author: Bob <davidwarro@gmail.com>
Date:   Fri Feb 24 15:02:15 2017 -0600

    Nueva idea

commit 66ef9145b9f12f222e419adc7942570010289df8
Author: davidphysdavalos <davidwarro@gmail.com>
Date:   Fri Feb 24 15:02:12 2017 -0600

    Se mejora el gitignore y se anade mas informcion a inicio.txt

commit 600fbcc03e46cdfffda5711d1b9ba2c9fbbb1b96
Author: davidphysdavalos <davidwarro@gmail.com>
Date:   Fri Feb 24 15:02:07 2017 -0600

    Commit inicial


To /mnt/homes/home/david/Desktop/original.git
   66ef914..ce94b43  master -> master


Ahora, veamos que es lo que Alice ve. 

In [23]:
;cd ../alice

/mnt/homes/home/david/Desktop/alice


In [24]:
;git pull

Updating 66ef914..ce94b43
Fast-forward
 idea_bob.txt | 1 +
 1 file changed, 1 insertion(+)
 create mode 100644 idea_bob.txt


From /mnt/homes/home/david/Desktop/original
   66ef914..ce94b43  master     -> origin/master


In [25]:
;ls

algo.tmp
idea_bob.txt
inicio.txt


Efectivamente, ahora Alice tiene la nueva idea de Bob 

### Alice y Bob hacen cambios en archivos diferentes

Vamos a hacer un cambio en un archivo de Alice, y a enviarlo al repositorio. 

Posteriormente intentaremos hacer lo mismo con Bob.

In [26]:
;date >> inicio.txt && git add . && git commit -m"Se anade la fecha" && git push

[master d68cd6f] Se anade la fecha
 1 file changed, 1 insertion(+)


To /mnt/homes/home/david/Desktop/original.git
   ce94b43..d68cd6f  master -> master


In [27]:
;cd ../bob

/mnt/homes/home/david/Desktop/bob


In [28]:
;date >> idea_bob.txt && git add . && git commit -m"Se anade otra fecha" && git push

[master ca086b7] Se anade otra fecha
 1 file changed, 1 insertion(+)


To /mnt/homes/home/david/Desktop/original.git
 ! [rejected]        master -> master (fetch first)
error: failed to push some refs to '/mnt/homes/home/david/Desktop/original.git'
hint: Updates were rejected because the remote contains work that you do
hint: not have locally. This is usually caused by another repository pushing
hint: to the same ref. You may want to first integrate the remote changes
hint: (e.g., 'git pull ...') before pushing again.
hint: See the 'Note about fast-forwards' in 'git push --help' for details.


Tenemos un problema. El cambio qeu queremos hacer ha sido rechazado [rejected]

De hecho, `git` nos está diciendo como resolver el problema. Debemos traer los cambios a nuestro repositorio mediante un `pull`.

In [29]:
;git pull

Merge made by the 'recursive' strategy.
 inicio.txt | 1 +
 1 file changed, 1 insertion(+)


From /mnt/homes/home/david/Desktop/original
   ce94b43..d68cd6f  master     -> origin/master


y automaticamente se hace un "merge" que combina los cambios de las dos partes.

Sin embargo, debemos hacer push para que los cambios, que se hicieron a nivel del repositorio de Bob, queden en el repositorio central. Similarmente, actualizamos a alice.

In [30]:
;git push

To /mnt/homes/home/david/Desktop/original.git
   d68cd6f..6aef0c9  master -> master


In [31]:
;cd ../alice

/mnt/homes/home/david/Desktop/alice


In [32]:
;git pull

Updating d68cd6f..6aef0c9
Fast-forward
 idea_bob.txt | 1 +
 1 file changed, 1 insertion(+)


From /mnt/homes/home/david/Desktop/original
   d68cd6f..6aef0c9  master     -> origin/master


In [33]:
;git log --graph --pretty

*   commit 6aef0c9f7a47b398c01983b2e80c1cfe0addb7a7
|\  Merge: ca086b7 d68cd6f
| | Author: Bob <davidwarro@gmail.com>
| | Date:   Fri Feb 24 15:02:20 2017 -0600
| | 
| |     Merge branch 'master' of /mnt/homes/home/david/Desktop/original
| |   
| * commit d68cd6f7d13c5d323c5ada1986f1343f00af6ce3
| | Author: davidphysdavalos <davidwarro@gmail.com>
| | Date:   Fri Feb 24 15:02:18 2017 -0600
| | 
| |     Se anade la fecha
| |   
* | commit ca086b7372d42cc84fd964e04ebf004b65e03b6a
|/  Author: Bob <davidwarro@gmail.com>
|   Date:   Fri Feb 24 15:02:19 2017 -0600
|   
|       Se anade otra fecha
|  
* commit ce94b43bab15d6408ec4d3e78233a65eec2a69e6
| Author: Bob <davidwarro@gmail.com>
| Date:   Fri Feb 24 15:02:15 2017 -0600
| 
|     Nueva idea
|  
* commit 66ef9145b9f12f222e419adc7942570010289df8
| Author: davidphysdavalos <davidwarro@gmail.com>
| Date:   Fri Feb 24 15:02:12 2017 -0600
| 
|     Se mejora el gitignore y se anade mas informcion a inicio.txt
|  
* commit 600fbcc03e46cdfffda571

### Cuando cambiamos el mismo archivo

Esto no debería pasar. Pero en la practica, nos sucede con cierta frecuencia, especialmente editando notas. 

Vamos a hacer cambios en las dos partes (Alice y Bob) y veamos como se comporta `git` en cada una y como resolverlo. Ambas partes editarán el archivo `inicio.txt`.

In [34]:
;pwd

/mnt/homes/home/david/Desktop/alice


In [35]:
;echo "comentario de alicia" >> inicio.txt && git add . && git commit -m"Otra idea de alicia" && git push

[master 12c8ffb] Otra idea de alicia
 1 file changed, 1 insertion(+)


To /mnt/homes/home/david/Desktop/original.git
   6aef0c9..12c8ffb  master -> master


In [36]:
;cd ../bob

/mnt/homes/home/david/Desktop/bob


In [37]:
;echo "comentario de bob" >> inicio.txt && git add . && git commit -m"Otra idea de bob"

[master f8b9955] Otra idea de bob
 1 file changed, 1 insertion(+)


In [38]:
;git push

To /mnt/homes/home/david/Desktop/original.git
 ! [rejected]        master -> master (fetch first)
error: failed to push some refs to '/mnt/homes/home/david/Desktop/original.git'
hint: Updates were rejected because the remote contains work that you do
hint: not have locally. This is usually caused by another repository pushing
hint: to the same ref. You may want to first integrate the remote changes
hint: (e.g., 'git pull ...') before pushing again.
hint: See the 'Note about fast-forwards' in 'git push --help' for details.


Naturalmente, los cambios de nuevo son rechazados. Intentamos hacer un pull, pero en esta ocasión tenemos un mensaje de error un poco mas severo. 

In [39]:
;git pull

Auto-merging inicio.txt
CONFLICT (content): Merge conflict in inicio.txt
Automatic merge failed; fix conflicts and then commit the result.


From /mnt/homes/home/david/Desktop/original
   6aef0c9..12c8ffb  master     -> origin/master


Este error indica que tenemos qeu resolver el conflicto en forma manual. Si nuestro archivo es código puro, como en este caso, no hay problema. Si es un archivo un poco mas complicado, típicamente toca escoger alguna de las versiones. En nuestro caso, podemos ver los dos contenidos del archivo:

In [40]:
;cat inicio.txt

iniciamos este archivo con perros y gatos
mas informacion
vie feb 24 15:02:18 CST 2017
<<<<<<< HEAD
comentario de bob
comentario de alicia
>>>>>>> 12c8ffbf5c58554745f9b54bc4f8d8b31cfaa627


Podemos editarlo manualmente para que ahora se vea asi:

In [41]:
;cat inicio.txt

iniciamos este archivo con perros y gatos
mas informacion
vie feb 24 15:02:18 CST 2017
comentario de bob
comentario de alicia


Y ya podemos hacer un commit y su posterior push.

In [42]:
;git add . && git commit -m"Hicimos un merge manual" && git push && git status

[master f21d4ff] Hicimos un merge manual
On branch master
Your branch is up-to-date with 'origin/master'.

nothing to commit, working directory clean


To /mnt/homes/home/david/Desktop/original.git
   12c8ffb..f21d4ff  master -> master


# Trabajando en ramas

El concepto de una *rama* ("branch") en git provee una forma sencilla y eficiente de trabajar en nuevas ideas, o de colaborar en un proyecto común, evitando romper cosas que a priori ya funcionan.

Para empezar, listemos las ramas existentes de un proyecto (por ejemplo, en el directorio `Alicia/`):
```
    > git branch
```
o usando
```
    > git branch -v
```
que brinda además el hash del último commit. Lo que esto indica es que existe únicamente la rama `master`, que es la rama que se crea por default (y en algún sentido es la principal), y el asterisco indica que estamos trabajando en esa rama.

Para crear una nueva rama, ejecutamos:
```
    > git branch <nombre_rama>
```
donde `<nombre_rama>` es el nombre de la rama, que es más o menos arbitrario y flexible. Un 
ejemplo es: `bugfix1`.

Después de ejecutar alguna de estas instrucciones, `git branch -v` nos informa que *ambas* ramas, `master` y `bugfix1` existen, ambas están en el (mismo) último commit, y el asterisco indica que estamos en la rama `master` aún.

Para cambiarnos de rama, ejecutamos:
```
    > git checkout <nombre_rama>
```
Nuevamente, existe un atajo para crear y cambiarnos de rama de un golpe: `git checkout -b <nombre_rama>`.

Ahora, la linea de status cobra mas sentido. 

In [43]:
;git checkout -b bugfix1 && git status

On branch bugfix1
nothing to commit, working directory clean


Switched to a new branch 'bugfix1'


Ahora podemos crear un archivo nuevo, y modificar uno existente. 

In [44]:
;echo "propuesta para arreglo de bug 1" >> inicio.txt && echo "nuevo codigo qeu lo corrige" >> arreglo_bug1.txt

In [45]:
;git add . && git commit -m"Primer intento de correccion"

[bugfix1 8dfc5e1] Primer intento de correccion
 2 files changed, 2 insertions(+)
 create mode 100644 arreglo_bug1.txt


In [46]:
;ls

arreglo_bug1.txt
idea_bob.txt
inicio.txt


In [47]:
;cat inicio.txt

iniciamos este archivo con perros y gatos
mas informacion
vie feb 24 15:02:18 CST 2017
comentario de bob
comentario de alicia
propuesta para arreglo de bug 1


In [48]:
;git checkout master

Your branch is up-to-date with 'origin/master'.


Switched to branch 'master'


In [49]:
;ls

idea_bob.txt
inicio.txt


In [50]:
;cat inicio.txt 

iniciamos este archivo con perros y gatos
mas informacion
vie feb 24 15:02:18 CST 2017
comentario de bob
comentario de alicia


El punto importante hasta el momento es que la historia de los dos branches (locales) ha divergido, y *ambas* historias están en ambas ramas.

Supongamos ahora que ya están satisfechos con los cambios que han hecho, después de muchas pruebas exhaustivas y otras fallidas (tal vez en otras ramas). Ahora queremos poner estos cambios en la rama `master`. Para esto, primero nos cambiamos a `master`, que es la rama a donde queremos pasar los cambios, y después hacemos un `merge`, o sea, fundimos las dos historias:
```
    > git checkout master   
    > git merge <nombre_rama>
```
En nuestro caso, podemos simplemente hacer un merge, de la misma manera que haciamos con otros repositorios. En este caso, lo podemos hacer directamente. 

In [51]:
;git checkout master && git merge bugfix1

Your branch is up-to-date with 'origin/master'.
Updating f21d4ff..8dfc5e1
Fast-forward
 arreglo_bug1.txt | 1 +
 inicio.txt       | 1 +
 2 files changed, 2 insertions(+)
 create mode 100644 arreglo_bug1.txt


Already on 'master'


De nuevo, un `git log` puede resultar particularmente util, si hemos hecho cambios en diferentes branches.

#  Facilitar el flujo de trabajo con GitHub

Para evitar que GitHub te esté pidiendo tu usuario y contraseña todo el tiempo, es necesario usar *claves de SSH* (SSH keys). En Linux y Mac, el procedimiento es como sigue. NB: **No** hacer esto desde una máquina/cuenta pública.

Utiliza el comando `ssh-keygen` para generar claves nuevas. Te pedirá que pongas una clave ("passphrase"); esta clave tendrás que ponerla sólo una vez por sesión.

In [52]:
;ssh-keygen

Generating public/private rsa key pair.


Enter file in which to save the key (/home/david/.ssh/id_rsa): 

Esto generará claves en el directorio escondido `~/.ssh` en tu directorio hogar:

In [53]:
;ls ~/.ssh

authorized_keys
config
config~
id_rsa
id_rsa.pub
known_hosts
known_hosts.old
sabe


Ahora, copia la clave pública; esto se puede hacer a mano (copiando el *contenido*  del archivo `id_rsa.pub`), o usando un programa. E.g. en Mac, puedes usar `pbcopy` para copiar el contenido de un archivo al clipboard:

In [54]:
;pbcopy < ~/.ssh/id_rsa.pub

/bin/bash: pbcopy: command not found


Ahora, hay que dar de alta las claves en GitHub:

- Ve a la página de tu cuenta en GitHub
- Escoge `Settings` (arriba, del lado derecho)
- Escoge `SSH keys`
- Escoge `Add SSH key`
- Pega lo que copiaste

Ya deberías poder hacer transacciones con GitHub sin que te pida tu usuario cada vez.


Una vez más, **no** hagas esto en un máquina o cuenta pública.

# Trabajar con un fork

Normalmente hacemos un **fork** de un repositorio de interés en GitHub, es decir, una copia del repositorio en tu propia cuenta de GitHub.

Al hacer `git clone ...` de tu fork, `git` provee un remote (es decir, un nombre para un repositorio remoto) llamado `origin`, que apunta a tu fork. Esto lo podemos ver con

In [62]:
;cd Desktop

LoadError: chdir Desktop: no such file or directory (ENOENT)

In [63]:
;git clone https://github.com/davidphysdavalos/MetodosNumericosAvanzados.git

Cloning into 'MetodosNumericosAvanzados'...


In [64]:
;cd MetodosNumericosAvanzados

/mnt/homes/home/david/Desktop/bob/MetodosNumericosAvanzados/MetodosNumericosAvanzados


In [65]:
; git remote -v

origin	https://github.com/davidphysdavalos/MetodosNumericosAvanzados.git (fetch)
origin	https://github.com/davidphysdavalos/MetodosNumericosAvanzados.git (push)


Vemos que `origin` apunta al fork del repositorio `MetodosNumericosAvanzados` en mi cuenta de GitHub (con usuario `dpsanders`).

Sin embargo, para mantener actualizado nuestro fork con respecto al repositorio original, debemos darle a conocer a `git` que también existe dicho repositorio. Si hacemos

In [66]:
;git help remote

GIT-REMOTE(1)                     Git Manual                     GIT-REMOTE(1)



NAME
       git-remote - manage set of tracked repositories

SYNOPSIS
       git remote [-v | --verbose]
       git remote add [-t <branch>] [-m <master>] [-f] [--[no-]tags] [--mirror=<fetch|push>] <name> <url>
       git remote rename <old> <new>
       git remote remove <name>
       git remote set-head <name> (-a | --auto | -d | --delete | <branch>)
       git remote set-branches [--add] <name> <branch>...
       git remote set-url [--push] <name> <newurl> [<oldurl>]
       git remote set-url --add [--push] <name> <newurl>
       git remote set-url --delete [--push] <name> <url>
       git remote [-v | --verbose] show [-n] <name>...
       git remote prune [-n | --dry-run] <name>...
       git remote [-v | --verbose] update [-p | --prune] [(<group> | <remote>)...]


DESCRIPTION
       Manage the set of repositories ("remotes") whose branches you track.

OPTIONS
       -v, --verbose
           Be a litt

vemos que hay un subcomando `add` de remote. Así que hacemos

In [67]:
; git remote add upstream https://github.com/dpsanders/MetodosNumericosAvanzados

El nombre usual que se le asigna al repositorio original es `upstream`.

Ahora podemos actualizar nuestro repositorio *local* con 

In [68]:
; git pull upstream master

Already up-to-date.


From https://github.com/dpsanders/MetodosNumericosAvanzados
 * branch            master     -> FETCH_HEAD
 * [new branch]      master     -> upstream/master


(el cual jala la rama `master` del repositorio apuntado por `upstream`).

Ahora al hacer

In [70]:
; git push

Everything up-to-date


empuja los cambios a `origin`, o sea, a nuestro propio fork.

* pull request: Una situación típica usando github, es que no tenemos los privilegios de hacer push directamente a *upstream*, lo que pódemos hacer es un *pull request*, es decir, desde nuestro fork (usado la pagina de github) le pedimos a los administradores del repositorio *upstream* que hagan un pull de nuestro fork.

* __Ejercicio:__ Hacer un fork del repositorio de la clase en su cuenta de github. Hacer un git clone de su fork y agregar un archivo con el formato que deben llevar las tareas, donde el contenido sea el nombre de su cerveza favorita o cualquier otra cosa.
    * Recordemos el formato de la tarea: T(numero)(nombre).extension, para este ejercicio será T0(Su_nombre).txt
    * Pasos:
        * Entrar a la página de github del curso y hacer un fork.
        * Hacer un clone del fork en su maquina.
        * Crear un archivo dentro de la carpeta tareas cuyo nombre sea T0(Su_nombre).txt
        * Hacer git _add, commit_ y _push_ a su fork, es decir:
            ```
            > git push
            ```
            lo que equivale a 
            ```
            > git push origin master
            ```
        * Una vez actualizado su fork en github, presionar el botón que dice *pull request* y seguir las instrucciones.
        * github mostrará las diferencias entre nuestor fork y el repositorio _upstream_ y nos dará la opción de documentar el pull request.
        * Una vez hecho el pull request, el administrador del repositorio _upstream_ decide si aceptar los cambios y hacer un merge de los dos repositorios.

# Tarea

* Agregar el origen _upstream_ al respositorio de la clase:
    Ahora que todos cuentan con un fork del repositorio de la clase, en la copia local del mismo, en su maquina, deberán agregar el origen del repositorio _oficial_ de la clase como lo muestra el ejemplo en este notebook. Es decir, deberán hacer:
    ```
    > git remote add upstream https://github.com/carlospgmat03/2017-2_TS_Pineda
    ```
    Al repositorio local de su fork, de esta forma podrán tener al dia los cambios hechos en las clases ejecutando
    ```
    > git pull upstram master
    ```
    sin que se les olvide actualizar su fork (que es el repositorio al cual si estan autorizados a hacer push directamente):
    ```
    > git push
    ```
    que es lo mismo que:
    ```
    > git push origin master
    ```
    Este es el mecanismo con el cual subiran las tareas, es decir, añadirán las nuevas tareas, en el formato correcto, dentro de la carpeta de tareas de su repositorio local. Luego harán _push_ a su fork y entonces un _pull request_.

# Para la siguiente iteracion

* Revisar que el git config funciona donde lo voy a hacer. 
* Poner los dibujitos explicitamente en el notebook

